This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
import json
with open('Api_key_NO_GIT.json', 'r') as f:
    j_data = json.load(f)
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = j_data.get('API_KEY')

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import pprint
pp = pprint.PrettyPrinter(indent=4)
from collections import OrderedDict, defaultdict, namedtuple

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
base_url = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json"
api_url = "&api_key="+API_KEY
url = base_url+"?limit=1"+api_url
r = requests.get(url)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
pp.pprint(r.json())

{   'dataset_data': {   'collapse': None,
                        'column_index': None,
                        'column_names': [   'Date',
                                            'Open',
                                            'High',
                                            'Low',
                                            'Close',
                                            'Change',
                                            'Traded Volume',
                                            'Turnover',
                                            'Last Price of the Day',
                                            'Daily Traded Units',
                                            'Daily Turnover'],
                        'data': [   [   '2018-06-07',
                                        None,
                                        60.85,
                                        59.65,
                                        59.95,
                                        No

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

# 1. 2017 data for the ticker AFX_X

In [5]:
afx_2017_url = base_url+"?start_date=2016-12-31&end_date=2018-01-01"+api_url
r = requests.get(afx_2017_url)

In [6]:
# columns in data
print(r.json()['dataset_data']['column_names'])
# first and last item in data confirming I have all 2017 information
print(r.json()['dataset_data']['data'][0])
print(r.json()['dataset_data']['data'][-1])

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']
['2017-12-29', 51.76, 51.94, 51.45, 51.76, None, 34640.0, 1792304.0, None, None, None]
['2017-01-02', 34.99, 35.94, 34.99, 35.8, None, 44700.0, 1590561.0, None, None, None]


# 2. Converting to a python dictionary

In [7]:
Tradeday = namedtuple('Tradeday', 'date open high low close change traded_vol turn_over last_price d_trad_unit d_turnover')

# one line conversion
data = {d[0] : Tradeday(*d) for d in r.json()['dataset_data']['data']}

# another way using a for loop
data2 = defaultdict(list) 
for d in r.json()['dataset_data']['data']:
    data2[d[0]] = Tradeday(*d)
    

# 3. Highest and lowest opening prices 

In [8]:
for v in data.values():
    if not v.open:
        print ('No listed open price due to public holiday on: {}'.format(v.date))
    
high = max(v.open for v in data.values() if v.open)
hi_date = [v for v in data.values() if v.open == high][0][0]
print('\nThe highest open price in 2017 was: {:.2f} on {}'.format(high, hi_date))

low = min(v.open for v in data.values() if v.open)
lo_date = [v for v in data.values() if v.open == low][0][0]
print('The lowest open price in 2017 was: {:.2f} on {}'.format(low, lo_date))


No listed open price due to public holiday on: 2017-05-01
No listed open price due to public holiday on: 2017-04-17
No listed open price due to public holiday on: 2017-04-14

The highest open price in 2017 was: 53.11 on 2017-12-14
The lowest open price in 2017 was: 34.00 on 2017-01-24


# 4. What was the largest change in any one day
based on high and low price.

In [9]:
p_range = max(v.high-v.low for v in data.values())
p_range_date = [v for v in data.values() if v.high-v.low == p_range][0][0]
print('The largest trading range in 2017 was: {:.2f} on {}'.format(p_range, p_range_date))


The largest trading range in 2017 was: 2.81 on 2017-05-11


# 5. Largest change between any two days?
Based on Closing Price

In [10]:
# Ensuring data is ordered by date
o_data = OrderedDict(sorted(data.items(), key=lambda t:t[0]))

In [16]:
%%timeit
# Method 1: flatten data into list of (k,v) tuples and iterate over a range
od_list = list(o_data.items())
max_change, max_st, max_ed  = 0, '', ''

for i in range(len(o_data)-1):
    change = abs(od_list[i][1].close-od_list[i+1][1].close)
    if change > max_change:
        max_change, max_st, max_ed = change, od_list[i][1].date, od_list[i+1][1].date

# Un comment to print all close prices for QA
#[print(i[1].date, i[1].close) for i in od_list] 

125 µs ± 9.67 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [15]:
print('The largest intraday price change was: {:.2f}'.format(max_change))
print('Between trading days {} and {}'.format(max_st, max_ed))

The largest intraday price change was: 2.56
Between trading days 2017-08-08 and 2017-08-09


In [17]:
from itertools import islice

In [21]:
%%timeit
# A one liner as I wanted to explore using islice and thought it should be possible
max_ch_ol = max(abs(v.close-next(islice(o_data.values(), i+1, i+2)).close) for i, v in enumerate(o_data.values()) if i < len(o_data)-1)
# A cleaner one line solution
max(abs(u.close-v.close) for u,v in zip(list(o_data.values()),list(o_data.values())[1:]))

1.57 ms ± 129 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [20]:
print('A one liner using islice: {:.2f}'.format(max_ch_ol))

A one liner using islice: 2.56


In [24]:
%%timeit
# Method 2: Avoiding creating a list in memory by using islice needed to loop to extract the dates.
mx, st, en = 0, '', ''
for i, v in enumerate(o_data.values()):
    if i < len(o_data)-1:
        ch = abs(v.close-next(islice(o_data.values(), i+1, i+2)).close)
        if ch > mx:
            mx, st, en = ch, v.date, next(islice(o_data.values(), i+1, i+2)).date


1.58 ms ± 84.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [23]:
print('The largest intraday price change was: {:.2f}'.format(mx))
print('Between trading days {} and {}'.format(st, en))

The largest intraday price change was: 2.56
Between trading days 2017-08-08 and 2017-08-09


In [27]:
%%timeit
# As the list seems faster trying the simpler loop interator - SLOWEST YET!.
mx1, st1, en1 = 0, '', ''
for i in range(len(o_data)-1):
    ch = abs(next(islice(o_data.values(), i, i+1)).close-next(islice(o_data.values(), i+1, i+2)).close)
    if ch > mx1:
         mx1, st1, en1 = ch, next(islice(o_data.values(), i, i+1)).date, next(islice(o_data.values(), i+1, i+2)).date


2.87 ms ± 52.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [28]:
print('The largest intraday price change was: {:.2f}'.format(mx1))
print('Between trading days {} and {}'.format(st1, en1))

The largest intraday price change was: 2.56
Between trading days 2017-08-08 and 2017-08-09


# 6. Average daily trading volume during this year?

In [29]:
vol_list = [v.traded_vol for v in data.values()]
ave_vol = sum(vol_list)/len(vol_list)
print('The ave trading volume in 2017 was: {:.2f}'.format(ave_vol))

ave_vol = 0
print('\nRunning calc:')
for idx, vol in enumerate(vol_list, 1):
    ave_vol = (ave_vol * (idx-1) + vol)/idx
    if idx % 20 == 0:
        print('    .. {} days of the year: {:.2f}'.format(idx,ave_vol))
print('Ave volume for all 2017 is {:.2f}'.format(ave_vol))

The ave trading volume in 2017 was: 89124.34

Running calc:
    .. 20 days of the year: 95779.60
    .. 40 days of the year: 83683.48
    .. 60 days of the year: 75834.52
    .. 80 days of the year: 76223.94
    .. 100 days of the year: 84965.96
    .. 120 days of the year: 86845.85
    .. 140 days of the year: 87302.20
    .. 160 days of the year: 84394.18
    .. 180 days of the year: 87820.66
    .. 200 days of the year: 95502.14
    .. 220 days of the year: 92269.95
    .. 240 days of the year: 90507.28
Ave volume for all 2017 is 89124.34


# 7. What was the median trading volume during this year?

In [30]:
def median(lst):
    qu, re = divmod(len(lst),2)
    srt = sorted(lst)
    if re:
        return srt[qu]
    return (srt[qu-1]+srt[qu])/2.

med = median(vol_list)
print('Median volume for 2017 is {:.2f}'.format(med))

Median volume for 2017 is 76286.00
